In [1]:
from perses.analysis import utils
from openmmtools.multistate import MultiStateReporter, MultiStateSamplerAnalyzer
from simtk import unit
import os
import itertools
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import logging
from perses.analysis import utils

from openmmtools.constants import kB
KT_KCALMOL = kB * 298 * unit.kelvin / unit.kilocalories_per_mole

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Given a path, get dg and ddg
def analyze(path):
    reporter = MultiStateReporter(path)
    analyzer = MultiStateSamplerAnalyzer(reporter)
    f_ij, df_ij = analyzer.get_free_energy()
    f = f_ij[0,-1] # units = kT
#     f = f_ij[0,-1] * analyzer.kT
#     dg = f.in_units_of(unit.kilocalories_per_mole)
    df = df_ij[0, -1]
#     df = df_ij[0, -1] * analyzer.kT
#     ddg = df.in_units_of(unit.kilocalories_per_mole)
    return f, df

def analyze_dom(path):
    reporter = MultiStateReporter(path)
    ncfile = utils.open_netcdf(path)
    n_iterations = ncfile.variables['last_iteration'][0]
    print(f"n_iterations: {n_iterations}")
    dg, ddg = list(), list()
#     iterations_to_analyze = list(range (1, n_iterations, 500)) + [int(n_iterations-1)]
#     print(iterations_to_analyze) # check that this is giving me the last avail iteration
#     for step in iterations_to_analyze: 
#         analyzer = MultiStateSamplerAnalyzer(reporter, max_n_iterations=step)
#         f_ij, df_ij = analyzer.get_free_energy()
#         dg.append(f_ij[0,-1])
#         ddg.append(df_ij[0,-1])
    # Get final free energy
    analyzer = MultiStateSamplerAnalyzer(reporter, max_n_iterations=n_iterations)
    f_ij, df_ij = analyzer.get_free_energy()
    dg.append(f_ij[0,-1])
    ddg.append(df_ij[0,-1])
    return np.array(dg), np.array(ddg)

### 5 ns N501Y -> N501Y/K417N (repex/31/9/0)

In [6]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [3]:
main_dir = 31
sub_dir = 9

In [4]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [6]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -144.75808851199668 (0.6315968946894134) kcal/mol
0 apo: -149.46330720968723 (0.2667006024049485) kcal/mol
0 DDG: 4.705218697690555 (0.685597439248771) kcal/mol



### 5 ns N501Y/K417N -> N501Y (repex/31/10/0)

In [5]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [7]:
main_dir = 31
sub_dir = 10

In [8]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [10]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -17.22130570739417 (0.4272927026616299) kcal/mol
0 apo: -13.448647206377055 (0.21610625122801985) kcal/mol
0 DDG: -3.772658501017114 (0.4788329203048889) kcal/mol



### 5 ns N501Y ->  N501Y/E484K (repex/31/11/0)

In [11]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [12]:
main_dir = 31
sub_dir = 11

In [13]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [14]:
# Read the dicts in and compute free energies (kT)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1] for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kT")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kT")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kT")
        print()


0 complex: -170.9137010483867 (1.098914353040906) kT
0 apo: -171.3724958724239 (0.7387483437513426) kT
0 DDG: 0.4587948240371986 (1.3241457135506896) kT



In [15]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -101.21284950585913 (0.6507626501088377) kcal/mol
0 apo: -101.48454177625351 (0.4374770687204254) kcal/mol
0 DDG: 0.27169227039438226 (0.7841417043066217) kcal/mol



### 5 ns N501Y/E484K -> N501Y (repex/31/12/0)

In [5]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [16]:
main_dir = 31
sub_dir = 12

In [17]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [18]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -226.8202980553889 (0.9015562979951257) kcal/mol
0 apo: -225.72246784969101 (0.5236596856019335) kcal/mol
0 DDG: -1.0978302056978748 (1.0426040594489319) kcal/mol



### 5 ns K417N -> N501Y/K417N (repex/31/13/0)

In [8]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [19]:
main_dir = 31
sub_dir = 13

In [20]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [21]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: 70.24930574910664 (0.5169836054301186) kcal/mol
0 apo: 64.6115695371746 (0.1493973089048564) kcal/mol
0 DDG: 5.637736211932037 (0.5381371611323061) kcal/mol



### 5 ns N501Y/K417N -> K417N (repex/31/14/0)

In [22]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [32]:
main_dir = 31
sub_dir = 14

In [33]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [34]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -66.96121367958516 (0.8172181073743694) kcal/mol
0 apo: -66.23151657480075 (0.2283117557225905) kcal/mol
0 DDG: -0.7296971047844067 (0.8485114570951168) kcal/mol



### 5 ns  K417N -> K417N/E484K (repex/31/15/0)

In [3]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [4]:
main_dir = 31
sub_dir = 15

In [36]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [37]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -100.84652476655265 (0.7445755888318999) kcal/mol
0 apo: -102.48307567995919 (0.36729252069205554) kcal/mol
0 DDG: 1.6365509134065377 (0.8302388832382488) kcal/mol



### 5 ns K417N/E484K -> K417N (repex/31/16/0)

In [5]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [6]:
main_dir = 31
sub_dir = 16

In [7]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [8]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -225.9310034092168 (1.1484197704874264) kcal/mol
0 apo: -227.32741481215442 (0.5704781507553002) kcal/mol
0 DDG: 1.3964114029376162 (1.2823077983602766) kcal/mol



### 5 ns E484K -> N501Y/E484K (repex/31/17/0)

In [29]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [46]:
main_dir = 31
sub_dir = 17

In [47]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex


n_iterations: 5000


analyzing apo
n_iterations: 5000


In [48]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: 70.82640385404905 (0.5933352750352326) kcal/mol
0 apo: 66.56474738081432 (0.3799065078843004) kcal/mol
0 DDG: 4.261656473234723 (0.7045393554188291) kcal/mol



### 5 ns N501Y/E484K -> E484K (repex/31/18/0)

In [49]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [50]:
main_dir = 31
sub_dir = 18

In [51]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex


n_iterations: 5000


analyzing apo
n_iterations: 5000


In [52]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -65.0266126337625 (0.6806472813233365) kcal/mol
0 apo: -65.97777758460128 (0.28919055388856285) kcal/mol
0 DDG: 0.9511649508387876 (0.7395349200891213) kcal/mol



### 5 ns E484K -> K417N/E484K (repex/31/19/0)

In [53]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [54]:
main_dir = 31
sub_dir = 19

In [55]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000


analyzing apo
n_iterations: 5000


In [56]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -149.18546268044602 (0.6629632181294111) kcal/mol
0 apo: -148.7366553597091 (0.3542413845958802) kcal/mol
0 DDG: -0.4488073207369325 (0.7516695997264432) kcal/mol



### 5 ns K417N/E484K -> E484K (repex/31/20/0)

In [57]:
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [58]:
main_dir = 31
sub_dir = 20

In [59]:
for r in tqdm_notebook([0]):
    print (f"replicate {r}")
    output_dir = f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/'
    
    print("analyzing complex")
    complex_path = os.path.join(output_dir, f"{r}_complex.nc")
    complex_dg, complex_ddg = analyze_dom(complex_path)
    print("analyzing apo")
    apo_path = os.path.join(output_dir, f"{r}_apo.nc")
    apo_dg, apo_ddg = analyze_dom(apo_path)

    results = np.array([complex_dg, complex_ddg, apo_dg, apo_ddg])
    with open(f'/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy', 'wb') as f:
        np.save(f, results)

  0%|          | 0/1 [00:00<?, ?it/s]

replicate 0
analyzing complex
n_iterations: 5000

analyzing apo
n_iterations: 5000


In [60]:
# Read the dicts in and compute free energies (kcal/mol)
d_results = {}
for r in [0]:
    with open(f"/data/chodera/zhangi/perses_benchmark/repex/{main_dir}/{sub_dir}/{r}/results.npy", "rb") as f:
        results = np.load(f, allow_pickle=True)
        d_results[r] = results
        complex_dg, complex_ddg, apo_dg, apo_ddg = [result[-1]*KT_KCALMOL for result in results]
        print(f"{r} complex: {complex_dg} ({complex_ddg}) kcal/mol")
        print(f"{r} apo: {apo_dg} ({apo_ddg}) kcal/mol")
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        print(f"{r} DDG: {binding_dg} ({binding_ddg}) kcal/mol")
        print()



0 complex: -18.487573013482606 (0.6529008091829787) kcal/mol
0 apo: -16.762720669768846 (0.3180490521764371) kcal/mol
0 DDG: -1.72485234371376 (0.7262469732963562) kcal/mol

